In [2]:
import numpy as np

In [3]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [4]:
from datasets import load_dataset

In [5]:
ds = load_dataset('yelp_review_full')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [7]:
ds['train'][20]

{'label': 3,
 'text': "A great townie bar with tasty food and an interesting clientele. I went to check this place out on the way home from the airport one Friday night and it didn't disappoint. It is refreshing to walk into a townie bar and not feel like the music stops and everyone in the place is staring at you - I'm guessing the mixed crowd of older hockey fans, young men in collared shirts, and thirtysomethings have probably seen it all during their time at this place. \\n\\nThe staff was top notch - the orders were somewhat overwhelming as they appeared short-staffed for the night, but my waitress tried to keep a positive attitude for my entire visit. The other waiter was wearing a hooded cardigan, and I wanted to steal it from him due to my difficulty in finding such a quality article of clothing.\\n\\nWe ordered a white pizza - large in size, engulfed in cheese, full of garlic flavor, flavorful hot sausage. An overall delicious pizza, aside from 2 things: 1, way too much grease

In [8]:
ds['train'][0]['text']

"dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."

In [9]:
ds['train'].features

{'label': ClassLabel(names=['1 star', '2 star', '3 stars', '4 stars', '5 stars'], id=None),
 'text': Value(dtype='string', id=None)}

In [10]:
import pandas as pd

ds_train = pd.DataFrame(ds['train'])
ds_test = pd.DataFrame(ds['test'])

In [11]:
ds_train.head()

,label,text
0,4,dr. goldberg offers everything i look for in a...
1,1,"Unfortunately, the frustration of being Dr. Go..."
2,3,Been going to Dr. Goldberg for over 10 years. ...
3,3,Got a letter in the mail last week that said D...
4,0,I don't know what Dr. Goldberg was like before...


In [12]:
ds_test.head()

,label,text
0,0,I got 'new' tires from them and within two wee...
1,0,Don't waste your time. We had two different p...
2,0,All I can say is the worst! We were the only 2...
3,0,I have been to this restaurant twice and was d...
4,0,Food was NOT GOOD at all! My husband & I ate h...


In [13]:
ds_train['label'].value_counts()

label
4    130000
1    130000
3    130000
0    130000
2    130000
Name: count, dtype: int64

In [14]:
ds_test['label'].value_counts()

label
0    10000
2    10000
1    10000
3    10000
4    10000
Name: count, dtype: int64

In [15]:
from datasets import Dataset, DatasetDict

# assign the splits
train = Dataset.from_pandas(ds_train)
test = Dataset.from_pandas(ds_test)

# reconstruct both datasets into a Dataset Dict object
new_ds = DatasetDict(
    {
         'train': train,
       'test': test
    }
)

# view the resulting dataset dict object
new_ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

### **Manual Application**
In this project task, we will manually apply the tokenization and text representation methods on our project dataset.


In [16]:
print("The shape of train set is :",new_ds['train'].shape)
print("The shape of test set is :",new_ds['test'].shape)

The shape of train set is : (650000, 2)
The shape of test set is : (50000, 2)


Dataset is equally divided among different classes of reviews

## Step 1: Choose the Method

Choosing TFIDF because it is less sparse and better for text analysis

## Step 2: Preprocessing

### Removing Puncuation

In [17]:
import string

punctuation = string.punctuation

def remove_punctuation(review):
    # Replace one or more consecutive newline characters (\n+) with a single space
    review = re.sub(r'\n+', ' ', review)

    # Replace punctuation characters with spaces, preserve other characters
    processed_review = ''.join([' ' if char in punctuation else char for char in review])

    # Ensure multiple spaces are reduced to a single space
    processed_review = re.sub(r'\s+', ' ', processed_review)

    return processed_review.strip()


### Text to lower case

In [18]:
def lower_case(review):
    return review.lower() if isinstance(review, str) else review

### Stop word removal

In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [21]:
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

### URLs / HyperLinks Removal

In [22]:
def remove_urls(review):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', review)

### Lemmatizer

In [23]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [24]:
#instantiating an object
lemmater = WordNetLemmatizer()

In [25]:
def lemmatize_text(review):
    review = [lemmater.lemmatize(word,'v') for word in review]
    review = [lemmater.lemmatize(word,'n') for word in review]
    return review

### Handling Contractions

In [26]:
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 10.6 MB/s eta 0:00:00


In [27]:
import contractions

In [28]:
def replace_contractions(review):
    return contractions.fix(review)

## Step 3: Tokenization

In [29]:
import re

def tokenization(review):
    return re.findall(r'\b\w+\b', review)

### Processing function

In [30]:
def preprocessing(review):
    review = remove_urls(review)
    review = replace_contractions(review)
    review = remove_punctuation(review)
    review = lower_case(review)
    tokens = tokenization(review)
    tokens = lemmatize_text(tokens)
    tokens = remove_stopwords(tokens)
    return ' '.join(tokens)  # Return as a single string

#### Sample check for our preprocessing function

In [31]:
print("Before preprocessing : ",new_ds['test'][10156]['text'])

check = preprocessing(new_ds['test'][10156]['text'])

print("After preprocessing : ",check)

Before preprocessing :  http://www.yelp.com/biz_photos/battistas-hole-in-the-wall-las-vegas?select=fp3gb-ENJa1b3pi-impq3A\n\nI have a friend who loves this place so much that she is having her wedding reception here. When she found out that we were heading to Las Vegas six weeks ago she insisted we had to check this place out. Attached is a picture of me outside the restaurant. Obviously it was taken before we dined because I am smiling.\n\nFormulaic beyond description! The wait staff have everything down to such a science that there is absolutely no warmth. They bring you free, undrinkable wine and point to the menu on the wall. You order from a handful of traditional and delicious-sounding Italian entrees, ranging in price from about $25.00 to about $50.00.  Immediately after ordering they bring a mealy iceberg salad in a wooden bowl, or soup, and some garlic bread which was fine. \n\nI had the chicken parmesan. It was clearly reconstituted, formed, frozen and reheated. I love Italia

# Apply preprocessing to the dataset

In [32]:
new_ds = new_ds.map(lambda x: {'text': preprocessing(x['text'])})

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

## Step 4: Vectorization

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

In [34]:
# Convert back to pandas DataFrame for further processing

train_df = new_ds['train'].to_pandas()
test_df = new_ds['test'].to_pandas()

In [35]:
train_df.head()

,label,text
0,4,dr goldberg offer everything look general prac...
1,1,unfortunately frustration dr goldberg patient ...
2,3,go dr goldberg 10 year think one 1st patient s...
3,3,get letter mail last week say dr goldberg move...
4,0,know dr goldberg like move arizona let tell st...


In [36]:
test_df.head()

,label,text
0,0,get new tire within two week get flat take car...
1,0,waste time two different people come house giv...
2,0,say worst 2 people place lunch place freeze lo...
3,0,restaurant twice disappoint time go back first...
4,0,food good husband eat couple week ago first ti...


#### Savings dataframes for further notebooks

In [37]:
train_df.to_csv('train_df.csv', index=False)
test_df.to_csv('test_df.csv', index=False)

### Sampling datasets for faster computation

In [38]:
train_df_small = train_df.sample(frac=0.1, random_state=42)
test_df_small = test_df.sample(frac=0.1, random_state=42)

In [39]:
# Fit and transform the train data
X_train = vectorizer.fit_transform(train_df_small['text'])

In [40]:
# Transform the test data (only transform)
X_test = vectorizer.transform(test_df_small['text'])

In [41]:
#labels
y_train = train_df_small['label']
y_test = test_df_small['label']

## ML Training and Testing
### Logistic Regression

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

In [43]:
model = LogisticRegression(multi_class='auto', max_iter=1000)

In [44]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [45]:
y_pred = model.predict(X_test)

In [46]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_mat)

Accuracy: 54.98%
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.71      0.69      1015
           1       0.48      0.46      0.47      1018
           2       0.45      0.44      0.45      1004
           3       0.48      0.48      0.48      1012
           4       0.65      0.67      0.66       951

    accuracy                           0.55      5000
   macro avg       0.55      0.55      0.55      5000
weighted avg       0.55      0.55      0.55      5000

Confusion Matrix:
[[722 222  46  15  10]
 [258 468 215  47  30]
 [ 66 223 441 225  49]
 [ 19  43 212 482 256]
 [ 23  10  57 225 636]]


### Random Forest

In [47]:
from sklearn.ensemble import RandomForestClassifier

In [48]:
rfc = RandomForestClassifier(random_state=42)

In [49]:
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [50]:
y_pred_rfc = rfc.predict(X_test)

In [51]:
# Accuracy
accuracy = accuracy_score(y_test, y_pred_rfc)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred_rfc))

# Confusion Matrix
conf_mat = confusion_matrix(y_test, y_pred_rfc)
print("Confusion Matrix:")
print(conf_mat)

Accuracy: 49.28%
Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.77      0.65      1015
           1       0.43      0.35      0.39      1018
           2       0.40      0.37      0.38      1004
           3       0.44      0.37      0.40      1012
           4       0.58      0.62      0.60       951

    accuracy                           0.49      5000
   macro avg       0.48      0.49      0.48      5000
weighted avg       0.48      0.49      0.48      5000

Confusion Matrix:
[[782 151  54  13  15]
 [353 355 204  63  43]
 [147 215 368 198  76]
 [ 54  69 220 371 298]
 [ 66  36  66 195 588]]


### MultiClass Naive Bayes

In [52]:
from sklearn.naive_bayes import MultinomialNB

In [53]:
mnb = MultinomialNB()

In [54]:
mnb.fit(X_train, y_train)

MultinomialNB()

In [55]:
mnb_predict = mnb.predict(X_test)

In [56]:
# Accuracy
accuracy = accuracy_score(y_test, mnb_predict)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Classification Report
print("Classification Report:")
print(classification_report(y_test, mnb_predict))

# Confusion Matrix
conf_mat = confusion_matrix(y_test, mnb_predict)
print("Confusion Matrix:")
print(conf_mat)

Accuracy: 49.64%
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.66      0.62      1015
           1       0.43      0.48      0.45      1018
           2       0.43      0.45      0.44      1004
           3       0.44      0.52      0.48      1012
           4       0.71      0.37      0.49       951

    accuracy                           0.50      5000
   macro avg       0.52      0.49      0.50      5000
weighted avg       0.52      0.50      0.50      5000

Confusion Matrix:
[[665 307  33   6   4]
 [245 487 226  52   8]
 [ 95 231 451 206  21]
 [ 41  75 258 526 112]
 [ 83  36  76 403 353]]
